### 데이터프레임의 결합
- 유니언 결합 
    - 단순하게 행을 결합하거나 열을 결합
    - pandas 안에 있는 concat()
        - 첫번째 인자 : 결합한 데이터프레임을 리스트 형태
        - axis 매개변수 : 0(행을 추가하는 결합), 1(열을 추가하는 결합)
        - ignore_index : False(기본값) (axis를 기준으로 행을 초기화 할것인가 열을 초기화할것인가 지정)
- 조인 결합
    - 특정한 조건에 맞춰서 데이터프레임을 열을 추가하는 결합 
    - 데이터프레임 + 데이터프레임 -> 특정 컬럼의 데이터들이 같은 값들로 이루어져있는 경우 열을 추가
    - pandas 안에 있는 merge()
        - 첫번째 인자 : 데이터프레임명
        - 두번째 인자 : 데이터프레임명 
        - on 매개변수 : 조건식(컬럼의 이름)
            - left_on : 첫번째 데이터프레임의 컬럼명
            - right_on : 두번째 데이터프레임의 컬럼명
        - how 매개변수 : left, right, inner, outer

In [ ]:
import pandas as pd

In [ ]:
data = {
    'name' : ['test', 'test2', 'test3'], 
    'age' : [20, 30, 40]
}
df1 = pd.DataFrame(data)
df1

In [ ]:
data2 = {
    'name' : ['test4', 'test5'], 
    'loc' : ['seoul', 'busan']
}
df2 = pd.DataFrame(data2)
df2

In [ ]:
# 유니언 결합 (단순한 행 결합)
pd.concat( [df1, df2], axis=0, ignore_index=True )

In [ ]:
# 유니언 결합 (단순한 열 결합)
pd.concat( [df1, df2], axis=1, ignore_index=True )

In [ ]:
data3 = {
    'name' : ['test6', 'test7'], 
    'age' : [25, 35], 
    'loc' : ['ulsan', 'sangju']
}
df3 = pd.DataFrame(data3)
df3

In [ ]:
# 3개의 데이터프레임을 결합(행 결합)
pd.concat( [df1, df2, df3], axis=0, ignore_index=True)

In [ ]:
data4 = {
    'id' : ['test', 'test2', 'test3', 'test4'], 
    'item' : ['A', 'B', 'A', 'D']
}
df4 = pd.DataFrame(data4)
df4

In [ ]:
data5 = {
    'item' : ['A', 'B', 'C'], 
    'price' : [1000, 2000, 3000]
}
df5 = pd.DataFrame(data5)
df5

In [ ]:
# 조인결합 (특정 조건에 맞춰서 열을 결합) : 2개의 데이터프레임을 결합
pd.merge( df4, df5, on = 'item', how='outer' )

In [ ]:
data6 = {
    'id' : ['test', 'test4', 'test3', 'test2'], 
    'name' : ['kim', 'park', 'lee', 'choi']
}
df6 = pd.DataFrame(data6)
df6

In [ ]:
# 조인결합을 이용하여 3개의 데이터프레임을 결합
# 조인결합은 2개의 데이터프레임만 결합이 가능하기 때문에 merge함수를 2번 사용
join_df = pd.merge(df4, df5, on='item', how='left')
join_df

In [ ]:
pd.merge(join_df, df6, on='id', how='right')

In [23]:
from glob import glob

In [ ]:
# 파일의 목록 로드 
file_list = glob('../csv/2017/*.csv')
file_list

In [29]:
# 비어있는 데이터프레임을 생성
df2017 = pd.DataFrame()
# file_list를 이용하여 반복문을 생성 
for file in file_list:
    # print(file)
    df = pd.read_csv(file)
    # df2017에 df 데이터를 추가(df2017과 df를 단순한 행 결합)
    df2017 = pd.concat([df2017, df], axis=0, ignore_index=True)

In [ ]:
df2017.info()

In [33]:
import os

In [38]:
## 특정 디렉토리의 데이터 파일들을 결합하는 함수 생성
# 매개변수
    # 특정 경로 (_path)
    # encoding engine (_encoding = 'UTF-8')
def dir_load(_path, _encoding = 'UTF-8'):
    # _path에서 확장자 부분만 따로 추출
    # 경로와 파일의 이름을 분리 
    # ex) ../csv/corona.csv -> (../csv/, corona.csv)
    dir, file_name = os.path.split(_path)
    # print(dir, file_name)
    # 파일의 이름에서 확장자를 분리
    # ex) corona.csv -> (corona, .csv)
    head, tail = os.path.splitext(file_name)
    # print(head, tail)

    # 비어있는 데이터프레임 생성
    result = pd.DataFrame()

    # _path를 이용하여 파일의 목록을 로드 
    file_list = glob(_path)
    # print(file_list)

    # file_list를 기준으로 반복문을 생성 
    for file in file_list:
        # tail이 csv라면 read_csv()를 이용
        if tail == '.csv':
            df = pd.read_csv(file, encoding=_encoding)
        # tail이 json이라면 read_json()을 이용
        elif tail == '.json':
            df = pd.read_json(file, encoding=_encoding)
        # tail이 xml이라면 read_xml()을 이용
        elif tail == '.xml':
            df = pd.read_xml(file, encoding=_encoding)
        # tail이 .xlsx이거나 .xls라면 read_excel()을 이용
        # elif (tail == '.xlsx') | (tail == '.xls'):
        elif tail in ['.xlsx', 'xls']:
            df = pd.read_excel(file)
        else:
            print('지원하지 않는 확장자입니다.')
            break
        # df를 result에 단순한 행 결합 
        result = pd.concat([result, df], axis=0)
    # result의 인덱스를 초기화하고 기존의 인덱스는 제거 
    result.reset_index(drop=True, inplace=True)
    # result를 되돌려준다
    return result

In [40]:
df_2017 = dir_load("../csv/2017/*.csv")

In [ ]:
df_2017.info()

In [42]:
df_2021 = dir_load("../csv/2021/*.json")

In [ ]:
df_2021.info()

In [ ]:
df_2021.head()

In [49]:
card_df = dir_load('../csv/card/*.csv', _encoding='cp949')

In [50]:
card_df

,crym,tco_btc_nm,ma_fem_dc,agegrp_dc,tco_btc_u_ct,tco_btc_u_am
0,201901,백화점,1,1,1,7.600000e+03
1,201901,백화점,1,10,379,1.945872e+07
2,201901,백화점,1,20,45173,4.747711e+09
3,201901,백화점,1,30,271858,2.791826e+10
4,201901,백화점,1,40,391109,3.791149e+10
...,...,...,...,...,...,...
3294,202109,NaN,2,50,3237324,1.813605e+11
3295,202109,NaN,2,60,1019719,7.002663e+10
3296,202109,NaN,2,70,159261,1.309103e+10
3297,202109,NaN,2,80,20094,1.935122e+09


In [51]:
# 모듈을 로드 
import data_load as dl

In [ ]:
dl.dir_load('../csv/2018/*.csv')